<a href="https://colab.research.google.com/github/moung1012/Numpy-Pandas/blob/master/Keras3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 영화 리뷰 분류 이진 분류 예제
- 인터넷 영화 데이터베이스로부터 가져온 양극단의 리뷰 5만개로 이루어진 데이터셋
- 훈련 데이터 2만 5000개와 테스트 데이터 2만 5000개로 나뉘어 있고 50%는 부정 50%는 긍정 리뷰로 구성되어 있다
- 같은 데이터에서 훈련하고 테스트 해서는 절대 안된다
- 중요한 것은 새로운 데이터에 대한 모델의 성능

In [ ]:
from keras.datasets import imdb
(train_data, train_labels), (test_data,test_labels) = imdb.load_data(num_words=10000)
# nom_words = 10000 매개 변수는 훈련 데이터에서 가장 자주 나타나는 단어 1만 개만 사용하겠다는 의미
# train_data와 test_data는 리뷰의 목록
# 각 리뷰의 단어 인덱스의 리스트(단어 시퀀스가 인코딩 된것)
#train_labels와 test_labels는 부정을 나타내는 0과 긍정을 나타내는 1의 리스트

In [ ]:
train_data[0]

In [ ]:
train_labels[0]

In [ ]:
max([max(sequence)] for sequence in train_data) # 단어 1만개로 제한했기 때문에 인덱스는 9,999

In [ ]:
word_index = imdb.get_word_index() # word_index는 단어와 정수 인덱스를 매핑한 딕셔너리
revers_word_index= dict(
    [(value, key) for (key, value) in word_index.items()]) # 정수 인덱스와 단어를 매핑하도록 뒤집는다
decoded_review = ' '.join(
    [reverse_word_index.get(i - 3,'?')for i in train_data[0]]) # 리뷰를 디코딩한다 0,1,2는 '패딩','문서 시작','사전에 없음'을 위한 인덱스이므로 3을 뺀다

### 데이터 준비  
리스트를 텐서로 바꾸는 두가지 방법  
- 같은 길이가 되도록 리스트에 패딩을 추가하고 (samples, sequence_length)크기의 정수 텐서로 변환 그 다음 정수 텐서를 다룰 수 있는 층을 신경망의 첫 번째 층으로 사용  
- 리시트를 원-핫 인코딩(one-hot incoding)하여 0과 1의 벡터로 변환한다

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # 크기가 (len(sequences), dimension))이고 모든 원소가 0인 행렬을 만듭니다
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # results[i]에서 특정 인덱스의 위치를 1로 만듭니다
    return results

# 훈련 데이터를 벡터로 변환합니다
x_train = vectorize_sequences(train_data)
# 테스트 데이터를 벡터로 변환합니다
x_test = vectorize_sequences(test_data)

In [ ]:
x_train[0]

In [ ]:
# 레이블을 벡터로 바꿉니다
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

### 신경망 모델 만들기

Dense 층을 쌓을 때 두가지 중요한 구조상의 결정이 필요하다  
- 얼마나 많은 층을 사용할 것인가
- 각 층에 얼마나 많은 은닉 유닛을 둘 것인가  
현재 모델의 구조  
- 16개의 은닉 유닛을 가진 2개의 은닉 층
- 현재 리뷰의 감정을 스칼라 값의 예측으로 출력하는 세 번째 층

다음이 이 신경망의 모습입니다:

![3-layer network](https://s3.amazonaws.com/book.keras.io/img/ch3/3_layer_network.png)

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
# 중간의 은닉층은 활성화 함수 relu를 사용하고 마지막 층은 확률을 출력하기 위해 시그모이드 활성화 함수 사용

---
마지막으로 손실 함수와 옵티마이저를 선택  
- 이진 분류 문제이고 신경망의 출력이 확률이기 때문에 binary_crossentropy 손실이 적합
- 확률을 출력하는 모델을 사용할 때는 크로스엔트로피가 최선의 선택
- 크로스엔트로피는 정보 이론 뷴야에서 온 개념으로 확률 분포 간의 차이를 측정한다(여기서는 원본 분포와 예측 분포 사이를 측정)


In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
#rmsprop 옵티마이저와 binary_crossentropy 손실 함수로 모델을 설정하는 단계

rmsprop, binary_crossentropy, accuracy가 포함되어 있기 때문에 옵티마이저, 손실 함수, 측정 지표를 문자열로 지정하는 것이 가능  
- 옵티마이저의 매개변수를 바꾸거나 자신만의 손실 함수, 측정 함수를 전달해야 할 경우 옵티마이저 파이썬 클래스를 사용해 직접 만들어 옵티마이저 매개변수에 전달

In [ ]:
from tensorflow.keras import optimizers

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras import losses
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 훈련 검증
- 훈련하는 동안 처음 본 데이터에 대한 모델의 정확도를 측정하기 위해서는 훈련 데이터에서 10,000의 샘프을 떼어서 검증 세트를 만들어야 한다

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

- 모델 512개 샘플씩 미니 배치를 만들어 20번의 에포크 동안 훈련 시킨다  
- 동시에 따로 떼어 놓은 만 개의 샘플에서 손실과 정확도를 측정해 보자
- 이렇게 하기 위해선 validation_data 매개변수에 검증 데이터를 전달 해야한다


In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

- model.fit() 메서드는 History 객체를 반환한다 

In [ ]:
history_dict = history.history
history_dict.keys()
#히스토리 객체는 훈련하는 동안 발생한 모든 정보를 담고있는 딕셔너리인 히스토리 속성을 가지고 있다
#이 딕셔너리는 훈련과 검증하는 동안 모니터링할 측정 지표당 하나씩 모두 내게의 항목을 담고 있다

맷플롯립을 사용해 훈련과 검증 데이터에 대한 손실과 정확도를 그려보자

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# ‘bo’는 파란색 점을 의미합니다
plt.plot(epochs, loss, 'bo', label='Training loss')
# ‘b’는 파란색 실선을 의미합니다
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # 그래프를 초기화합니다
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

- 점선은 훈련 손실과 정확도
- 실선은 검증 손실과 정확도

- 훈련 손실이 에포크마다 감소하고 훈련 정확도는 에포크마다 증가한다  
- 2번째 에포크 이후부터 훈련 데이터에 과도하게 최적화 되어 훈련 데이터에 특화된 표현을 학습하므로 훈련 세트 이외의 데이터에서는 일반화 하지 못한다
- 이런 경우 과대 적합을 방지하기 위해서 3번째 에포크 이후에 훈련을 중지할 수 있다

In [ ]:
#새로운 신경망을 4번의 에포크 동안만 훈련하고 테스트 데이터에서 평가해 보자
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
results #단순한 방식으로도 87%의 정확도를 달성했다

### 훈련된 모델로 새로운 데이터에 대해 예측하기
- predict 메서드를 사용해서 어떤 리뷰가 긍정일 확률을 예측할 수 있다

In [ ]:
model.predict(x_test)

모델은 어떤 샘플에 대해 확신을 가지고 있지만 어떤 샘플에 대해서는 확신이 부족하다

### 추가 실험
- 은닉층 1개 또는 3개의 은닉 층을 사용하고 검증과 테스트 정확도에 어떤 영향을 미치는지 확인해보기
- 층의 은닉 유닛을 추가하거나 줄여보자 32개 또는 64개
- mse손실 함수 사용해보기
- relu 대신에 tanh 활성화 함수 사용해 보기

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='tanh', input_shape=(10000,)))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

In [ ]:

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
results #손실 함수가 엄청나다 

### 정리
- 원본 데이터를 신경망에 텐서로 주입하기 위해서는 꽤 많은 전처리가 필요하다
- relu 활성화 함수와 함께 Dense층을 쌓은 네트워크는 앞으로 자주 사용하게 될 것이다
- 이진 분류 문제에서 네트워크는 하나의 유닛과 sigmoid 활성화 함수를 가진 Dense층으로 끝나야 한다. 이 신경망의 출력은 확률을 나타내는 0과 1사이의 스칼라 값
- rmsporp 옵티마이저는 문제에 상관없이 일반적으로 충분히 좋은 선택이다
- 훈련 데이터에 대해 성능이 향상됨에 따라 신경망은 과대적합되기 시작하고 이전에 본적 없는 데이터에서는 결과가 점점 나빠지게 된다. 항상 훈련 세트 이외에 데이터에서 성능을 모니터링 해야한다